## Predicting with Actual Testing Data using all the Selected ML models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
import pandas as pd
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
import re
import string
from nltk.corpus import stopwords

def clean_text(text):

    # Convert all to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))

    cleaned_words = [word for word in words if word.lower() not in stop_words]

    return ' '.join(cleaned_words)

# Apply the cleaning function to the column
train_df['text'] = train_df['text'].apply(clean_text)

In [ ]:
import re
import string
from nltk.corpus import stopwords

def clean_text(text):

    #convert all to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))

    cleaned_words = [word for word in words if word.lower() not in stop_words]

    return ' '.join(cleaned_words)

# Apply the cleaning function to the column
test_df['text'] = test_df['text'].apply(clean_text)

In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,happened terrible car crash
1,2,NaN,NaN,heard earthquake different cities stay safe ev...
2,3,NaN,NaN,forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,apocalypse lighting spokane wildfires
4,11,NaN,NaN,typhoon soudelor kills china taiwan


In [ ]:
X_train = train_df['text']
y_train = train_df['target']
X_train

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       people receive wildfires evacuation orders cal...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                                 utckm volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [ ]:
X_test = test_df['text']
X_test

0                             happened terrible car crash
1       heard earthquake different cities stay safe ev...
2       forest fire spot pond geese fleeing across str...
3                   apocalypse lighting spokane wildfires
4                     typhoon soudelor kills china taiwan
                              ...                        
3258    earthquake safety los angeles ûò safety faste...
3259    storm ri worse last hurricane cityampothers ha...
3260                        green line derailment chicago
3261             meg issues hazardous weather outlook hwo
3262    cityofcalgary activated municipal emergency pl...
Name: text, Length: 3263, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors
import numpy as np

word2vec_path = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
word_vectors = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

vectorizer = TfidfVectorizer()

train_tfidf_matrix = vectorizer.fit_transform(X_train)
test_tfidf_matrix = vectorizer.transform(X_test)

train_sum_tfidf_per_doc = train_tfidf_matrix.sum(axis=1)
test_sum_tfidf_per_doc = test_tfidf_matrix.sum(axis=1)

epsilon = 1e-10

normalized_train_tfidf_matrix = train_tfidf_matrix / (train_sum_tfidf_per_doc + epsilon)
normalized_test_tfidf_matrix = test_tfidf_matrix / (test_sum_tfidf_per_doc + epsilon)

In [ ]:
train_text_embeddings = []
for text in X_train:
    tokens = text.lower().split()
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    text_embedding = np.mean(embeddings, axis=0) if embeddings else None
    train_text_embeddings.append(text_embedding)

test_text_embeddings = []
for text in X_test:
    tokens = text.lower().split()
    embeddings = [word_vectors.get_vector(word) for word in tokens if word in word_vectors.key_to_index]
    text_embedding = np.mean(embeddings, axis=0) if embeddings else None
    test_text_embeddings.append(text_embedding)

In [ ]:
import numpy as np

# Assuming test_text_embeddings contains your embeddings
# Find the maximum dimension across all embeddings
max_dimension = max(embedding.shape[0] for embedding in train_text_embeddings if embedding is not None)

# Create an empty array to hold the standardized embeddings
train_standardized_embeddings = np.zeros((len(train_text_embeddings), max_dimension))

# Iterate through embeddings and adjust dimensions
for i, embedding in enumerate(train_text_embeddings):
    if embedding is not None:
        # Get the current dimension of the embedding
        current_dimension = embedding.shape[0]

        # Pad or truncate embeddings to match the maximum dimension
        if current_dimension < max_dimension:
            # Pad the embeddings with zeros to match the maximum dimension
            padding = np.zeros((max_dimension - current_dimension,))
            train_standardized_embeddings[i] = np.concatenate((embedding, padding))
        elif current_dimension > max_dimension:
            # Truncate the embeddings to match the maximum dimension
            train_standardized_embeddings[i] = embedding[:max_dimension]
        else:
            train_standardized_embeddings[i] = embedding

# Check the shape of the standardized embeddings array
print("Shape of standardized embeddings:", train_standardized_embeddings.shape)

Shape of standardized embeddings: (7613, 300)


In [ ]:
import numpy as np

# Assuming test_text_embeddings contains your embeddings
# Find the maximum dimension across all embeddings
max_dimension = max(embedding.shape[0] for embedding in test_text_embeddings if embedding is not None)

# Create an empty array to hold the standardized embeddings
test_standardized_embeddings = np.zeros((len(test_text_embeddings), max_dimension))

# Iterate through embeddings and adjust dimensions
for i, embedding in enumerate(test_text_embeddings):
    if embedding is not None:
        # Get the current dimension of the embedding
        current_dimension = embedding.shape[0]

        # Pad or truncate embeddings to match the maximum dimension
        if current_dimension < max_dimension:
            # Pad the embeddings with zeros to match the maximum dimension
            padding = np.zeros((max_dimension - current_dimension,))
            test_standardized_embeddings[i] = np.concatenate((embedding, padding))
        elif current_dimension > max_dimension:
            # Truncate the embeddings to match the maximum dimension
            test_standardized_embeddings[i] = embedding[:max_dimension]
        else:
            test_standardized_embeddings[i] = embedding

# Check the shape of the standardized embeddings array
print("Shape of standardized embeddings:", test_standardized_embeddings.shape)

Shape of standardized embeddings: (3263, 300)


In [ ]:
# Determine the minimum number of samples
min_samples = min(normalized_train_tfidf_matrix.shape[0], train_standardized_embeddings.shape[0])

# Trim or slice the larger dataset to match the smaller number of samples
trimmed_train_tfidf_matrix = normalized_train_tfidf_matrix.toarray()[:min_samples]
trimmed_train_embeddings = train_standardized_embeddings[:min_samples]
print(trimmed_train_tfidf_matrix.shape)
print(trimmed_train_embeddings.shape)

(7613, 16845)
(7613, 300)


In [ ]:
# Determine the minimum number of samples
min_samples = min(normalized_test_tfidf_matrix.shape[0], test_standardized_embeddings.shape[0])

# Trim or slice the larger dataset to match the smaller number of samples
trimmed_test_tfidf_matrix = normalized_test_tfidf_matrix.toarray()[:min_samples]
trimmed_test_embeddings = test_standardized_embeddings[:min_samples]
print(trimmed_test_tfidf_matrix.shape)
print(trimmed_test_embeddings.shape)

(3263, 16845)
(3263, 300)


In [ ]:
# Combine TF-IDF vectors with word embeddings for training, validation, and testing sets
combined_train_vectors = np.hstack((trimmed_train_tfidf_matrix, trimmed_train_embeddings))
combined_test_vectors = np.hstack((trimmed_test_tfidf_matrix, trimmed_test_embeddings))

In [ ]:
# Check the shapes of combined_train_val_vectors and train_val_labels after reduction
print("Updated Shape of combined_train_vectors:", combined_train_vectors.shape)
print("Original Shape of train_labels:", y_train.shape)

# Match the labels to the reduced number of samples in the combined vectors
if combined_train_vectors.shape[0] != y_train.shape[0]:
    # Adjust train_val_labels to match the reduced number of samples
    train_labels = y_train[:combined_train_vectors.shape[0]]
# Recheck the shapes after adjusting the labels
print("Updated Shape of train_labels:", y_train.shape)

Updated Shape of combined_train_vectors: (7613, 17145)
Original Shape of train_labels: (7613,)
Updated Shape of train_labels: (7613,)


### Multinomial Naive Bayes

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
combined_train_vectors_scaled = scaler.fit_transform(combined_train_vectors)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(combined_train_vectors_scaled, y_train)

# Predict labels for the actual testing set
predicted_labels = classifier.predict(combined_test_vectors)
print("Predicted Labels for Testing Set:", predicted_labels)

Predicted Labels for Testing Set: [1 0 1 ... 1 1 1]


### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(combined_train_vectors, y_train)

# Predict labels for the actual testing set
predicted_labels = classifier.predict(combined_test_vectors)
print("Predicted Labels for Testing Set:", predicted_labels)

Predicted Labels for Testing Set: [1 1 1 ... 1 1 1]


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(combined_train_vectors, y_train)

# Predict labels for the actual testing set
predicted_labels = classifier.predict(combined_test_vectors)
print("Predicted Labels for Testing Set:", predicted_labels)

Predicted Labels for Testing Set: [1 0 1 ... 1 1 1]


###KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(combined_train_vectors, y_train)

# Predict labels for the actual testing set
predicted_labels = classifier.predict(combined_test_vectors)
print("Predicted Labels for Testing Set:", predicted_labels)

Predicted Labels for Testing Set: [1 1 1 ... 1 1 1]


###Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier()
classifier.fit(combined_train_vectors, y_train)

# Predict labels for the actual testing set
predicted_labels = classifier.predict(combined_test_vectors)
print("Predicted Labels for Testing Set:", predicted_labels)

Predicted Labels for Testing Set: [1 1 1 ... 1 1 1]


### SVM

In [ ]:
from sklearn import svm
classifier = svm.SVC(kernel='linear')
classifier.fit(combined_train_vectors, y_train)

# Predict labels for the actual testing set
predicted_labels = classifier.predict(combined_test_vectors)
print("Predicted Labels for Testing Set:", predicted_labels)

Predicted Labels for Testing Set: [1 1 1 ... 1 1 0]


### Saving the Result matching with the Sample Submission format for Kaggle
The file names are changed each time for the respective ML models' predicted results

In [ ]:
# Match the number of samples in predicted_labels with X_test
if predicted_labels.shape[0] != len(X_test):
    # Adjust X_test to match the number of samples in predicted_labels
    X_test = X_test[:predicted_labels.shape[0]]

# Create a DataFrame with text and predicted labels
result_df_nb_kaggle = pd.DataFrame({'text': X_test, 'predicted_labels': predicted_labels})

# Display the DataFrame
result_df_nb_kaggle

,text,predicted_labels
0,happened terrible car crash,1
1,heard earthquake different cities stay safe ev...,1
2,forest fire spot pond geese fleeing across str...,1
3,apocalypse lighting spokane wildfires,1
4,typhoon soudelor kills china taiwan,1
...,...,...
3258,earthquake safety los angeles ûò safety faste...,1
3259,storm ri worse last hurricane cityampothers ha...,1
3260,green line derailment chicago,1
3261,meg issues hazardous weather outlook hwo,1


In [ ]:
import pandas as pd

# Save the DataFrame to a CSV file
result_df_nb_kaggle.to_csv('test_nb_kaggle.csv', index=False)

In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

In [ ]:
sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
sample_submission["target"] = predicted_labels

In [ ]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [ ]:
sample_submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
import pandas as pd

# Save the DataFrame to a CSV file
sample_submission.to_csv('sample_submission.csv', index=False)